In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

C:\Programmingtools\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#生成graph图后打开命令行，首先移动到logs文件夹所在的硬盘，之后执行命令
#tensorboard --logdir=C:\Users\Yannis\TensorFlow\logs(logs的绝对路径)，
#用浏览器访问复制生成的网址(进来用google浏览器)


# 载入数据集
mnist = input_data.read_data_sets("MNIST_data",one_hot=True) #把MNIST_data存在文件当前目录下，否者写明绝对路径

#每个批次的大小，训练模型时以批次为单位放入图片
batch_size = 100 #每次训练放入100张图片
#计算一共多少批次
n_batch = mnist.train.num_examples // batch_size  # // 代表整除

#参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean) #平均值   scalar方法可以记录一个值，并赋予这个值一个名字
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev) #标准差
        tf.summary.scalar('max', tf.reduce_max(var)) #最大值
        tf.summary.scalar('min', tf.reduce_min(var)) #最小值
        tf.summary.histogram('histogram', var) #直方图
#命名空间
with tf.name_scope('input'):     #命名为input
    #定义两个placeholder
    x = tf.placeholder(tf.float32,[None,784],name='x-input') 
    y = tf.placeholder(tf.float32,[None,10],name='y-input') 

#因为我们加了layer这个命名空间重新执行后生成的graph是在我们第一回生成的graph基础上生成的，
#导致会产生两个input命名，所以需要我们通过Kernel的Restart & Run All来重新执行生成我们需要的graph
with tf.name_scope('layer'):
    #创建一个简单的神经网络,只有输入层和输出层
    with tf.name_scope('weights'):
        W = tf.Variable(tf.zeros([784,10]),name='W') #每张图片有784个像素点，输入到神经网络中是一张一张输入的
        variable_summaries(W)
    with tf.name_scope('biases'):
        b = tf.Variable(tf.zeros([10]),name='b')
        variable_summaries(b)
    with tf.name_scope('Wx_plus_b_L'):
        Wx_plus_b_L = tf.matmul(x,W) + b
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(Wx_plus_b_L)

#交叉熵代价函数
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=Wx_plus_b_L))
    tf.summary.scalar('loss', loss)
with tf.name_scope('train'):  
    #使用梯度下降
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

#初始化变量
#不用起名字，有一个默认的名字init
init = tf.global_variables_initializer()

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        #结果存放在一个布尔型列表中
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1)) #比较两个参数是否相等，返回boolean型，argmax可以返回最大的数值在第几个位置，1表示按行查找
    #argmax返回一维张量中最大的值所在的位置 https://blog.csdn.net/u013580539/article/details/79339250
    with tf.name_scope('accuracy'):    
        #求准确率
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32)) #把boolean型转换成float型，并求平均值
        tf.summary.scalar('accuracy', accuracy)
                             
#合并所有的summary
merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('logs/',sess.graph)   #将graph图写入当前目录下的logs文件夹(没有便自动创建)里
    for epoch in range(10):   #把所有的图片训练21次
         for batch in range(n_batch):
                batch_xs, batch_ys = mnist.train.next_batch(batch_size) #图片的数据保存在batch_xs, 图片的标签保存在batch_ys，自动获取下一批次
                summary, _ = sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys}) #一边训练,一边统计
         
         writer.add_summary(summary,epoch) #每次epoch，记一次                  
         acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
         print("Iter" + str(epoch) + "Testing Accuracy" + str(acc))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Iter0Testing Accuracy0.9084
Iter1Testing Accuracy0.9169
Iter2Testing Accuracy0.9173
Iter3Testing Accuracy0.9173
Iter4Testing Accuracy0.9213
Iter5Testing Accuracy0.9202
Iter6Testing Accuracy0.9215
Iter7Testing Accuracy0.9243
Iter8Testing Accuracy0.9218
Iter9Testing Accuracy0.9215


In [3]:
# #另一种方法
# for i in range(200):
#     #每个批次100个样本
#     batch_xs, batch_ys = mnist.train.next_batch(100)
#     summary,_ = sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys})
#     writer.add_summary(summary,i)
#     if i % 500 == 0:
#         print(sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels}))